In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
print(os.listdir("../input"))

['item_metadata.csv', 'submission_popular.csv', 'test.csv', 'train.csv']


# train

In [19]:
train = pd.read_csv('../input/train.csv')
print('train data shape : ',train.shape)

train data shape :  (15932992, 12)


In [21]:
#unix to datetime
train['timestamp'] = pd.to_datetime(train['timestamp'],unit='s')
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,008UN2RY0DMY,cd10940538005,2018-11-01 00:14:03,1,search for destination,"Rochester, USA",US,"Rochester, USA",desktop,NaN,NaN,NaN
1,008UN2RY0DMY,cd10940538005,2018-11-01 00:17:43,2,change of sort order,interaction sort button,US,"Rochester, USA",desktop,NaN,NaN,NaN
2,008UN2RY0DMY,cd10940538005,2018-11-01 00:17:53,3,change of sort order,distance and recommended,US,"Rochester, USA",desktop,NaN,NaN,NaN
3,008UN2RY0DMY,cd10940538005,2018-11-01 00:17:53,4,filter selection,Focus on Distance,US,"Rochester, USA",desktop,Focus on Distance,NaN,NaN
4,008UN2RY0DMY,cd10940538005,2018-11-01 00:19:10,5,interaction item image,57103,US,"Rochester, USA",desktop,NaN,NaN,NaN


## 데이터 기준 날짜
- 2018-11-01 00:00:08~ 2018-11-06 23:59:59

In [71]:
print('train data 시작 시간 : ',train.timestamp.min())
print('train data 마지막 시간 : ',train.timestamp.max())

train data 시작 시간 :  2018-11-01 00:00:08
train data 마지막 시간 :  2018-11-06 23:59:59


## na

In [6]:
train.isnull().sum()

user_id                   0
session_id                0
timestamp                 0
step                      0
action_type               0
reference                 0
platform                  0
city                      0
device                    0
current_filters    15163375
impressions        14346406
prices             14346406
dtype: int64

In [15]:
print('prices NA인 경우와 impressions NA인 경우가 같다 ') if len(train[(train.prices.isna()) & (train.impressions.isna())]) == train.prices.isna().sum() else print('다르다!')

prices NA인 경우와 impressions NA인 경우가 같다 


### current_filters notna
- hotel ES 가 뭘까??

In [25]:
train.loc[train.current_filters.notna()].head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
3,008UN2RY0DMY,cd10940538005,2018-11-01 00:17:53,4,filter selection,Focus on Distance,US,"Rochester, USA",desktop,Focus on Distance,NaN,NaN
810,0A9DDRRPHSU8,0db28996deb66,2018-11-01 11:12:23,2,filter selection,Hostal (ES),US,"Barcelona, Spain",desktop,Hotel|Resort|Hostal (ES),NaN,NaN
811,0A9DDRRPHSU8,0db28996deb66,2018-11-01 11:12:23,3,filter selection,Hotel,US,"Barcelona, Spain",desktop,Hotel|Resort|Hostal (ES),NaN,NaN
812,0A9DDRRPHSU8,0db28996deb66,2018-11-01 11:12:23,4,filter selection,Resort,US,"Barcelona, Spain",desktop,Hotel|Resort|Hostal (ES),NaN,NaN
813,0A9DDRRPHSU8,0db28996deb66,2018-11-01 11:12:33,5,filter selection,Serviced Apartment,US,"Barcelona, Spain",desktop,Hotel|Resort|Hostal (ES)|House / Apartment|Ser...,NaN,NaN


- current_filters  가 na가 아닌 것들 중에서 action_type의 첫번째 값을 보면, filter selection이 아닐까?
-> 아님! 왜일까..

In [28]:
filter_ = pd.DataFrame(train.loc[train.current_filters.notna()].groupby('session_id')['action_type'].first())

In [30]:
#poi : points of interest(관심 지점 : gps)
filter_.action_type.value_counts()

filter selection          196572
search for destination      6752
search for poi              2342
clickout item                  5
change of sort order           2
interaction item image         2
search for item                1
Name: action_type, dtype: int64

In [35]:
train[train.session_id == filter_[filter_.action_type == 'clickout item'].index[0]]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
889435,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:15:53,1,clickout item,24361,IT,"Siena, Italy",tablet,NaN,4783948|5828800|8413364|150624|24366|2289794|2...,150|162|108|130|88|95|149|170|126|95|110|87|84...
889436,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:15:53,2,clickout item,24361,IT,"Siena, Italy",tablet,Free WiFi (Combined)|Pet Friendly|Breakfast In...,24337|150624|5828800|2397775|24366|3215782|243...,126|130|162|141|88|87|87|95|167|108|150|95|84|...
889437,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:21:36,3,search for destination,"Siena, Italy",IT,"Siena, Italy",tablet,Free WiFi (Combined)|Pet Friendly|Breakfast In...,NaN,NaN
889438,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:22:21,4,search for item,6278310,IT,"Siena, Italy",tablet,NaN,NaN,NaN
889439,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:22:24,5,interaction item image,6278310,IT,"Siena, Italy",tablet,NaN,NaN,NaN
889440,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:22:24,6,interaction item image,6278310,IT,"Siena, Italy",tablet,NaN,NaN,NaN
889441,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:22:28,7,interaction item image,6278310,IT,"Siena, Italy",tablet,NaN,NaN,NaN
889442,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:22:30,8,interaction item image,6278310,IT,"Siena, Italy",tablet,NaN,NaN,NaN
889443,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:22:30,9,interaction item image,6278310,IT,"Siena, Italy",tablet,NaN,NaN,NaN
889444,98CA5EEVX83K,2cc7ed7efc41b,2018-11-04 20:22:39,10,interaction item image,6278310,IT,"Siena, Italy",tablet,NaN,NaN,NaN


## count

### user ID & session ID

In [53]:
train.session_id.value_counts().head()

3167404ed3197    3522
948641e533837    2816
9233fb83c116b    2800
191ae48e3cb8e    2648
c9b863c921a2d    2640
Name: session_id, dtype: int64

In [38]:
print('user ID 개수 : ', len(train.user_id.unique()))
print('session ID 개수 : ', len(train.session_id.unique()))

user ID 개수 :  730803
session ID 개수 :  910683


In [46]:
#user_id 별 session_id 개수의 평균.
train.groupby(['user_id'])['session_id'].nunique().mean()

1.2461402046789627

### current_filters

In [20]:
train.current_filters.value_counts()

Sort by Price                                                                                                                                  77826
Focus on Distance                                                                                                                              42332
Best Value                                                                                                                                     40861
5 Star|Hotel|Motel|Resort|Hostal (ES)                                                                                                          24216
5 Star|4 Star|Hotel|Motel|Resort|Hostal (ES)                                                                                                   20014
5 Star|4 Star|3 Star|Hotel|Motel|Resort|Hostal (ES)                                                                                            16784
Excellent Rating                                                                                          

In [22]:
print('current_filters unique 개수 : ',len(train.current_filters.value_counts()))

current_filters unique 개수 :  60986


In [29]:
from collections import Counter

In [30]:
filter_count = [] #filter 개수 저장

for val in train.current_filters.value_counts().index :
    filter_count.append(val.count('|') + 1)
    
Counter(filter_count)

Counter({1: 188,
         5: 11756,
         6: 8217,
         7: 5492,
         2: 2758,
         9: 3003,
         8: 3972,
         3: 9201,
         10: 1901,
         4: 12948,
         11: 886,
         14: 81,
         12: 364,
         15: 40,
         13: 150,
         17: 7,
         18: 3,
         16: 19})

# test

In [50]:
test = pd.read_csv('../input/test.csv')
print('test data shape : ',test.shape)

test data shape :  (3782335, 12)


In [51]:
#unix to datetime
test['timestamp'] = pd.to_datetime(test['timestamp'],unit='s')
test.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,000324D9BBUC,89643988fdbfb,2018-11-07 12:21:12,1,search for destination,"Budapest, Hungary",BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
1,000324D9BBUC,89643988fdbfb,2018-11-07 12:30:20,2,filter selection,Car Park,BG,"Budapest, Hungary",desktop,Car Park,NaN,NaN
2,000324D9BBUC,89643988fdbfb,2018-11-07 12:31:22,3,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
3,000324D9BBUC,89643988fdbfb,2018-11-07 12:31:22,4,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN
4,000324D9BBUC,89643988fdbfb,2018-11-07 12:31:42,5,interaction item image,1000915,BG,"Budapest, Hungary",desktop,NaN,NaN,NaN


## 데이터 기준 날짜
- 2018-11-07 00:00:07 ~ 2018-11-08 23:59:59

In [72]:
print('test data 시작 시간 : ',test.timestamp.min())
print('test data 마지막 시간 : ',test.timestamp.max())

test data 시작 시간 :  2018-11-07 00:00:07
test data 마지막 시간 :  2018-11-08 23:59:59


## na

In [40]:
test.isnull().sum()

user_id                  0
session_id               0
timestamp                0
step                     0
action_type              0
reference           261731
platform                 0
city                     0
device                   0
current_filters    3570444
impressions        3253556
prices             3253556
dtype: int64

In [52]:
print('reference NA 개수 : ', test.reference.isnull().sum(), 'NA 비율', test.reference.isnull().sum()/len(test))

reference NA 개수 :  261731 NA 비율 0.06919825980512039


## user  ID & session ID

In [57]:
print('user ID 개수 : ', len(test.user_id.unique()))
print('session ID 개수 : ', len(test.session_id.unique()))

user ID 개수 :  250852
session ID 개수 :  291381


In [58]:
#user_id 별 session_id 개수의 평균.
test.groupby(['user_id'])['session_id'].nunique().mean()

1.1615653851673498

# train & test

## user_id & session_id

In [66]:
print('train & test의 user_id 가 겹치지 않는다.')if len(set(train.user_id) - set(test.user_id)) == 0 else print('train & test의 user_id가 겹친다.')
print('\n')
print('train에만 있는 user_id 개수',len(set(train.user_id) - set(test.user_id)))
print('test에만 있는 user_id 개수',len(set(test.user_id) - set(train.user_id)))
print('train, test 모두 포함하는 user_id 개수', len(set(train.user_id) & set(test.user_id)))

train & test의 user_id가 겹친다.


train에만 있는 user_id 개수 697189
test에만 있는 user_id 개수 217238
train, test 모두 포함하는 user_id 개수 33614


In [67]:
print('train & test의 session_id 가 겹치지 않는다.')if len(set(train.session_id) - set(test.session_id)) == 0 else print('train & test의 session_id가 겹친다.')
print('\n')
print('train에만 있는 session_id 개수',len(set(train.session_id) - set(test.session_id)))
print('test에만 있는 session_id 개수',len(set(test.session_id) - set(train.session_id)))
print('train, test 모두 포함하는 session_id 개수', len(set(train.session_id) & set(test.session_id)))

train & test의 session_id가 겹친다.


train에만 있는 session_id 개수 910667
test에만 있는 session_id 개수 291365
train, test 모두 포함하는 session_id 개수 16


- 겹치는 session_id 
- 2018.11.06에서 2018.11.07로 넘아가는 시점에도 이용한 경우!
- 또는 session 이 reset되지 않은 경우

In [74]:
set(train.session_id) & set(test.session_id)

{'2480cd59859f7',
 '2a181b2125efe',
 '35e9a348c9d07',
 '48880df1f1ac9',
 '54418e6c20dd6',
 '5ac1377bb44ec',
 '5ea73da580d41',
 '711fc00536723',
 '7e5ec0a512233',
 'a14268acc47ab',
 'a7255a848e9df',
 'a940556420c16',
 'a9bd5353ae089',
 'b76440eac54b3',
 'd8198666e22d6',
 'e6eb492282abf'}

In [ ]:
df = pd.concat([train,test])

In [75]:
#5일과 8일의 session_id가 같음
df.loc[df.session_id == '2480cd59859f7']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
10889022,AWKK3A3WA4VF,2480cd59859f7,2018-11-05 15:40:55,1,search for destination,"Cananéia, Brazil",BR,"Cananéia, Brazil",desktop,NaN,NaN,NaN
10889023,AWKK3A3WA4VF,2480cd59859f7,2018-11-05 15:41:50,2,search for item,9529066,BR,"Cananéia, Brazil",desktop,NaN,NaN,NaN
10889024,AWKK3A3WA4VF,2480cd59859f7,2018-11-05 15:42:12,3,search for destination,"Cananéia, Brazil",BR,"Cananéia, Brazil",desktop,NaN,NaN,NaN
10889025,AWKK3A3WA4VF,2480cd59859f7,2018-11-05 15:42:25,4,change of sort order,interaction sort button,BR,"Cananéia, Brazil",desktop,NaN,NaN,NaN
10889026,AWKK3A3WA4VF,2480cd59859f7,2018-11-05 15:42:34,5,change of sort order,price and recommended,BR,"Cananéia, Brazil",desktop,NaN,NaN,NaN
10889027,AWKK3A3WA4VF,2480cd59859f7,2018-11-05 15:42:34,6,filter selection,Best Value,BR,"Cananéia, Brazil",desktop,Best Value,NaN,NaN
10889028,AWKK3A3WA4VF,2480cd59859f7,2018-11-05 15:42:43,7,clickout item,1966469,BR,"Cananéia, Brazil",desktop,NaN,2273504|1966469|8997046|3795926|9605704|287654...,65|65|76|47|33|45|34|95|25|39|45|47|32
10889029,AWKK3A3WA4VF,2480cd59859f7,2018-11-05 15:58:54,8,search for destination,"New York, USA",BR,"New York, USA",desktop,NaN,NaN,NaN
1359283,AWKK3A3WA4VF,2480cd59859f7,2018-11-08 19:38:30,1,interaction item image,2777713,BR,"São Sebastião, Brazil",desktop,NaN,NaN,NaN
1359284,AWKK3A3WA4VF,2480cd59859f7,2018-11-08 19:38:30,2,interaction item image,2777713,BR,"São Sebastião, Brazil",desktop,NaN,NaN,NaN


### train과 test를 합쳤을 때

In [64]:
df.shape

(19715327, 12)

In [76]:
print('user ID 개수 : ', len(df.user_id.unique()))
print('session ID 개수 : ', len(df.session_id.unique()))

user ID 개수 :  948041
session ID 개수 :  1202048


In [77]:
#user_id 별 session_id 개수의 평균.
df.groupby(['user_id'])['session_id'].nunique().mean()

1.2679282858019854

# item_meta

In [80]:
item_metadata = pd.read_csv('../input/item_metadata.csv')

In [81]:
print('item_metadata data shape : ',item_metadata.shape)

item_metadata data shape :  (927142, 2)


In [51]:
item_metadata.head()

,item_id,properties
0,5121,Terrace (Hotel)|Lift|Childcare|From 3 Stars|Ra...
1,5555,Terrace (Hotel)|Lift|From 3 Stars|Radio|From 4...
2,5684,Terrace (Hotel)|Lift|From 3 Stars|Radio|4 Star...
3,5777,Lift|Childcare|From 3 Stars|Radio|4 Star|From ...
4,6044,Terrace (Hotel)|Childcare|From 3 Stars|Radio|B...


In [53]:
print('item metadata는 item_id로 유니크한 값을 갖는다') if len(item_metadata.item_id.unique()) == len(item_metadata) else print('not unique')

item metadata는 item_id로 유니크한 값을 갖는다


In [54]:
item_prop_count = [] #properties 개수 저장


for val in item_metadata.properties.value_counts().index :
    item_prop_count.append(val.count('|') + 1)
    
Counter(item_prop_count)

Counter({1: 37,
         2: 606,
         4: 4057,
         3: 1996,
         5: 7738,
         8: 9911,
         9: 10877,
         13: 10842,
         12: 10983,
         7: 9834,
         11: 11474,
         10: 11682,
         6: 10468,
         14: 10599,
         17: 12119,
         20: 13452,
         18: 12395,
         21: 13968,
         19: 12968,
         22: 14170,
         16: 11427,
         15: 10930,
         31: 13083,
         29: 14034,
         24: 14562,
         32: 12777,
         30: 13550,
         23: 14369,
         44: 7758,
         25: 14707,
         28: 14088,
         63: 2116,
         35: 11421,
         34: 11861,
         52: 4978,
         55: 3987,
         49: 5914,
         58: 3255,
         33: 12389,
         85: 237,
         60: 2702,
         59: 2962,
         51: 5332,
         45: 7233,
         27: 14352,
         41: 8686,
         38: 10104,
         26: 14528,
         53: 4548,
         42: 8498,
         37: 10346,
         79: 5

# submission popular

In [4]:
sub = pd.read_csv(path + 'submission_popular.csv')

In [5]:
sub.head()

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,924795 106315 1033140 119494 101758 903037 105...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,3505150 3812004 2227896 2292254 3184842 222702...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,4476010 3505150 3812004 2227896 2292254 222702...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22854 3067559 22721 22713 16121 22772 22727 22...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,9857656 5849628 655716 1352530 502066 1405084 ...


In [6]:
sub.shape

(253573, 5)

In [8]:
len(sub.user_id.unique())

250852

In [11]:
len(set(train.user_id) - set(sub.user_id))

697189

In [13]:
len(set(sub.user_id) - set(train.user_id))

217238

In [15]:
len(set(test.user_id) - set(sub.user_id))

0

In [14]:
len(set(sub.user_id) - set(test.user_id))

0

In [36]:
test.loc[test.session_id == 'e45f3c80fc652']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
2795238,PVECQNOF5NCR,e45f3c80fc652,1541551997,1,search for destination,"Cancun, Mexico",MX,"Cancun, Mexico",mobile,NaN,NaN,NaN


In [37]:
test.loc[test.session_id == '7f5f9a596640c']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
1255788,9UW1WZ0WPSE6,7f5f9a596640c,1541595135,1,search for item,1457493,BR,"Florianópolis, Brazil",mobile,NaN,NaN,NaN
1255789,9UW1WZ0WPSE6,7f5f9a596640c,1541595238,2,interaction item deals,3378782,BR,"Florianópolis, Brazil",mobile,NaN,NaN,NaN
1255790,9UW1WZ0WPSE6,7f5f9a596640c,1541595240,3,clickout item,3378782,BR,"Florianópolis, Brazil",mobile,NaN,1457493|1510229|1993165|477921|478616|1189712|...,154|172|90|121|224|190|85|64|143|183|75|167|11...
1255791,9UW1WZ0WPSE6,7f5f9a596640c,1541595438,4,clickout item,152438,BR,"Florianópolis, Brazil",mobile,NaN,1457493|1510229|1993165|477921|478616|1189712|...,154|172|90|121|224|190|85|64|143|183|75|167|11...
1255792,9UW1WZ0WPSE6,7f5f9a596640c,1541595585,5,interaction item deals,1253461,BR,"Florianópolis, Brazil",mobile,NaN,NaN,NaN
1255793,9UW1WZ0WPSE6,7f5f9a596640c,1541595588,6,clickout item,1253461,BR,"Florianópolis, Brazil",mobile,NaN,1241433|479006|1474801|1253461|1750539|1411164...,77|106|194|61|117|171|178|119|65|234|105|272|1...


In [34]:
set(test.session_id) - set(sub.session_id)

{'e45f3c80fc652',
 '7f5f9a596640c',
 '16a7078e8a46f',
 '54bfa4ab1a6e8',
 '695173b906898',
 'dd406e7e3516a',
 '6db044a90fb10',
 'f96fd6e39a339',
 '44e2a25c3db36',
 '3e54e8ca45afd',
 'cbd5abe0cd8f1',
 'c12956877512a',
 'd23c1cdbcab6c',
 '4c3efa2541af1',
 '0a0349606f3dd',
 'ce049b2c90471',
 '02b9e22bd5de5',
 '498522eb3e70b',
 '58e6f79bcee70',
 'a77ce95d8efd3',
 '96365535a478a',
 'e2f56a0fb6078',
 'e35be68e30beb',
 'b640b945b7757',
 '808bffa03ace8',
 '0fa1429c4aee8',
 'f89b76366e2e8',
 '281e7c4e4c74c',
 '15f856a2d1ede',
 '72f92df866937',
 'c5eb9245817a0',
 '6dba9c1f72383',
 '47fee8eb862a0',
 '052cd601e9757',
 '8878b002f501c',
 'bb5d0d2430435',
 'd336895f48ca4',
 '47976bc9718ee',
 '65a61635de2e4',
 '26532b91ffa3e',
 '83954cbee6eed',
 '5d302b7b59205',
 '3f91de0c54e60',
 'b511f23da0a1b',
 '9d377d7c08251',
 'cce5cd7760f33',
 'd6e77caf7a128',
 '02b0fc0d41b0e',
 '0ef06d9477a2e',
 '99efb2dd587ed',
 '769f875160c28',
 '80f70179580f3',
 '84e25b1f28d57',
 'bb8ee30e2c10f',
 'f71dd8f2adf18',
 'f74f4ef4

In [32]:
len(set(sub.session_id) - set(test.session_id))

0

In [24]:
pd.DataFrame(sub.groupby(['user_id'])['session_id'].count())

,session_id
user_id,
000324D9BBUC,1
0004Q49X39PY,2
00071784XQ6B,1
0008BO33KUQ0,1
000GO9NY6P4M,1
000IRHJS2DL9,1
000JB0UNEH23,1
000OWRCYEHKT,1
000VBY1D6BP8,1


In [26]:
pd.DataFrame(sub.groupby(['user_id'])['session_id'].count()).sort_values(by = 'session_id', ascending=False)

,session_id
user_id,
EJTNZ75DSSUH,5
2AWNQVAE8SIT,5
I4XJ1CQXAHP9,5
H94J10XWGJQK,5
5JE53WBX74DR,4
N6E1EY7GP33O,4
NSMT2HE49DGD,4
WVL6JGF5HTVT,4
IFO2OZY2GIO1,4


In [27]:
sub.loc[sub.user_id == 'EJTNZ75DSSUH']

,user_id,session_id,timestamp,step,item_recommendations
114444,EJTNZ75DSSUH,3126e7e283a25,1541714191,1,352416 5193540 7179650 1952071 2628297 105658 ...
114445,EJTNZ75DSSUH,574b04ff34580,1541590550,1,7126898 5193540 7179650 1952071 7875360 97748 ...
114446,EJTNZ75DSSUH,873239d77533b,1541621705,1,7126898 5193540 7179650 1952071 97748 2628297 ...
114447,EJTNZ75DSSUH,a777af410d7c5,1541625683,1,7126898 5193540 7179650 97748 2628297 105658 2...
114448,EJTNZ75DSSUH,e12c19568a7e3,1541657474,1,5193540 7179650 1952071 97748 2628297 105658 3...


In [28]:
test.loc[test.user_id == 'EJTNZ75DSSUH']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
1711886,EJTNZ75DSSUH,3126e7e283a25,1541714191,1,clickout item,NaN,TH,"Wanhua District, Taiwan",mobile,NaN,1040738|5193540|7179650|2628297|95185|105658|1...,89|76|82|61|198|93|115|62|123|468|177|192|211|...
1711887,EJTNZ75DSSUH,574b04ff34580,1541590550,1,clickout item,NaN,TH,"Wanhua District, Taiwan",mobile,NaN,1040738|5193540|7179650|1236765|95185|128403|8...,89|75|83|116|197|111|62|202|178|123|176|195|87...
1711888,EJTNZ75DSSUH,873239d77533b,1541621705,1,clickout item,NaN,TH,"Wanhua District, Taiwan",mobile,NaN,1040738|5193540|7179650|2628297|95185|105658|1...,89|76|83|61|197|92|110|62|123|421|177|196|222|...
1711889,EJTNZ75DSSUH,a777af410d7c5,1541625683,1,clickout item,NaN,TH,"Wanhua District, Taiwan",mobile,NaN,1040738|5193540|7179650|2628297|95185|105658|1...,89|76|83|61|197|92|110|62|421|227|177|196|222|...
1711890,EJTNZ75DSSUH,e12c19568a7e3,1541657474,1,clickout item,NaN,TH,"Wanhua District, Taiwan",mobile,NaN,1040738|5193540|7179650|2628297|95185|105658|1...,89|76|82|61|198|92|115|52|123|421|177|196|222|...


In [30]:
sub.loc[sub.user_id == 'YLZKXGCT2TIR']

,user_id,session_id,timestamp,step,item_recommendations
244723,YLZKXGCT2TIR,4533912ee1a82,1541601159,1,39597 1104666 84035 127320 39588 2643984 39574...
244724,YLZKXGCT2TIR,68e4fd99dbdf3,1541584870,1,39588 1492187 39574 1388968 4521406 41651 3958...
244725,YLZKXGCT2TIR,7148fd9e31487,1541631470,1,32580 39549 1490395 39511 39546 1492187 137194...


In [31]:
test.loc[test.user_id == 'YLZKXGCT2TIR']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
3650186,YLZKXGCT2TIR,4533912ee1a82,1541601159,1,clickout item,NaN,NL,"Portimão, Portugal",tablet,NaN,127320|1967651|39597|95267|39588|41678|84035|9...,30|25|40|43|83|79|42|36|85|72|15|43|58|25|55|6...
3650187,YLZKXGCT2TIR,68e4fd99dbdf3,1541584870,1,clickout item,NaN,NL,"Lagoa, Portugal",tablet,NaN,148940|41651|2366776|39588|39652|95480|1967651...,64|127|72|83|38|85|25|77|36|85|38|58|43|53|100...
3650188,YLZKXGCT2TIR,7148fd9e31487,1541631470,1,clickout item,NaN,NL,"Albufeira, Portugal",tablet,NaN,131815|1448879|39511|32580|39558|39549|1490395...,34|36|25|60|31|34|32|137|17|66|35|73|144|85|35...


In [ ]:
test.loc[test.user_id == 'YLZKXGCT2TIR']